In [2]:
# coding: utf-8
"""
simrankを用いた劣モジュラ最適化の検証
"""
from filer2.filer2 import Filer
from library.graphsubmodular import Polarity_GraphSubModular
from library.rouge import Rouge_N
from library.slackbot import SlackBot
import numpy as np
import glob

In [3]:
start = 0
end = 10

list_test_path = glob.glob('./files/rakuten_ichiba/dumpfile/test/*')
list_ans_path = glob.glob('./files/rakuten_ichiba/dumpfile/ans/*')

list_test_path.sort()
list_ans_path.sort()

list_test_path = list_test_path[start:end]
list_ans_path = list_ans_path[start:end]

print len(list_test_path)
print len(list_ans_path)

10
10


In [4]:
username='プログラム1-1'

counter = 1
try:
    for test_path, ans_path in zip(list_test_path, list_ans_path):
        dict_test = Filer.readdump(test_path)
        dict_ans = Filer.readdump(ans_path)
        filename = ans_path.replace('./files/rakuten_ichiba/dumpfile/ans/', '')
        filename = filename.replace('.dump', '')
        print counter
        counter += 1
        # テスト用ファイルの作成
        # １００単語以上ある文は除く
        list_sentence = []
        list_sep = []
        list_lemmas = []
        for row in dict_test:
            sen = row['sentence']
            sepall = row['sep_all']
            lemmas = row['sep_all_lemmas']
            if len(sepall) <= 100 and len(sepall) != 0:
                list_sentence.append(sen)
                list_sep.append(sepall)
                list_lemmas.append(lemmas)

        # インスタンス化
        PGSM = Polarity_GraphSubModular(list_sep=list_sep,
                                        list_sep_lemmas=list_lemmas,
                                        dict_path='./files/dictionary/dict_word_pol.dump',
                                        weighted=True,
                                        threshold=0.95,
                                        identity=0.4,
                                        mode='cor')
        for a, b in [[1.0, 0.0], [0.9, 0.1], [0.8, 0.2],
                     [0.7, 0.3], [0.6, 0.4], [0.5, 0.5],
                     [0.4, 0.6], [0.3, 0.7], [0.2, 0.8],
                     [0.1, 0.9], [0.0, 1.0]]:
            # 学習
            PGSM.m_greedy(num_w = 100, r=1, scale=0, a=a, b=b)
            # Rougeを計算するようにbag_of_wordsを作成する
            list_C = PGSM.list_C
            list_gm_test = [list_sep[row[0]] for row in list_C]
            list_gm_ans = dict_ans['sep_all']
            try:
                R1 = Rouge_N.rouge(list_gm_test, list_gm_ans, N=1)
            except:
                R1 = 0
            try:
                R2 = Rouge_N.rouge(list_gm_test, list_gm_ans, N=2)
            except:
                R2 = 0
            try:
                R3 = Rouge_N.rouge(list_gm_test, list_gm_ans, N=3)
            except:
                R3 = 0
            try:
                R4 = Rouge_N.rouge(list_gm_test, list_gm_ans, N=4)
            except:
                R4 = 0
            list_result = [[filename, 0, 1, a, b, R1, R2, R3, R4]]
            Filer.writecsv(list_result, './files/rakuten_ichiba/result/PGSM/all_%s_%s.csv'%(a, b))
    SlackBot.post_message(username=username,
                          text='%s号機、無事帰還しました！！' % username,
                          icon_emoji=':thumbsup:')
except:
    SlackBot.post_message(username=username,
                          text='エラー、下手くそ！！',
                          icon_emoji=':troll:')
    raise

1


NameError: name 'sep' is not defined